<a href="https://colab.research.google.com/github/juliosdutra/Controle_de_Processos_UFES/blob/main/3.Transfer-functions/simulacao_sistema_segunda_ordem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<left><img src="https://github.com/juliosdutra/Controle_de_Processos_UFES/blob/main/imagens/cabe%C3%A7alho-geral.jpg?raw=true" width="700" height="175">
</left>

# Simulação de Sistema de Segunda Ordem

Nesta aplicação, simula-se a resposta ao degrau de um **sistema de segunda ordem** para observar como os parâmetros do sistema afetam seu comportamento. Sistemas de segunda ordem são comuns em engenharia, especialmente em controle de processos e sistemas dinâmicos. Esses sistemas são descritos por três parâmetros principais:

1. **Ganho (K)**
2. **Período (τ)**
3. **Fator de Amortecimento (ξ)**

Durante a simulação, é possível ajustar os valores de **ganho (K)**, **período (τ)** e **fator de amortecimento (ξ)**. Ao mudar esses valores, observe como a curva de resposta muda. Dessa forma, é possível entender como esses parâmetros impactam a dinâmica de sistemas de segunda ordem. Isso é fundamental para o projeto e controle de sistemas dinâmicos, permitindo otimizar a resposta conforme as necessidades de diferentes processos.

In [138]:
# @title
from ipywidgets import interact, widgets, Button, Layout, VBox, Output
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from IPython.display import display, HTML


# Função para plotar a resposta ao degrau e exibir propriedades
def simula_2ordem(ganho, tau, fator_xi):
    K = ganho
    xi = fator_xi
    num = [K]  # Numerador da função de transferência
    den = [tau**2, 2*xi*tau, 1]  # Denominador da função de transferência

    if fator_xi < 1:
      omega_n = 1/tau # Frequência natural
      omega_d = omega_n*np.sqrt(1-fator_xi**2) # Frequência de amortecimento

      # propriedades
      sigma = fator_xi*omega_n
      beta = np.arctan(omega_d/sigma)
      t_rise = (np.pi-beta)/omega_d # tempo de subida
      t_peak = np.pi*tau/np.sqrt(1-fator_xi**2) # tempo de pico
      overshoot = 100*np.exp(-np.pi*fator_xi/np.sqrt(1-fator_xi*fator_xi)) # Overshoot (%)
      settling_time = 4*tau/fator_xi # Critério de 2% para settling time
      periodo_osc = 2*np.pi*tau/np.sqrt(1-fator_xi*fator_xi) # Período de oscilação
      y_assentamento = ganho


    # Criando a função de transferência usando scipy.signal.TransferFunction
    system = signal.TransferFunction(num, den)

    # Obtendo a resposta ao degrau
    if fator_xi<1:
      t, y = signal.step(system, T=np.linspace(0,(1.5 + fator_xi)*settling_time,500))
    else:
      t, y = signal.step(system)
      steady_state_value = y[-1]  # Valor em regime estacionário
      settling_threshold = 0.02  # Critério de 2% para settling time
      settling_time_idx = np.where(np.abs((y - steady_state_value)/steady_state_value) < settling_threshold)[0]
      settling_time = t[settling_time_idx[0]] if len(settling_time_idx) > 0 else np.nan
      y_assentamento = y[settling_time_idx[0]] if len(settling_time_idx) > 0 else np.nan

    # Cálculo do pico de resposta
    pico_resposta = (1 + overshoot / 100)*ganho  if fator_xi < 1 else ganho
    t_pico_resposta = t_peak if fator_xi < 1 else t[-1]
    t_subida_resposta = t_rise if fator_xi < 1 else settling_time

    # Plotando a resposta
    plt.figure(figsize=(8, 5))
    label_text = rf"$G(s) = \frac{{{num[0]}}}{{{round(den[0], 2)}s^2 + {round(den[1], 2)}s + {den[2]}}}$"
    plt.plot(t, y, color='dodgerblue', linewidth=2, label=label_text)
    plt.plot([t_pico_resposta], [pico_resposta], "ro", label='Tempo de pico')
    if fator_xi<1:
      #plt.plot([t_subida_resposta], [ganho], "sg", label='Tempo de subida')
      settling_y_idx = np.where((t - settling_time) > 0)[0]
      y_assentamento = y[settling_y_idx[0]] if len(settling_y_idx) > 0 else np.nan
      #plt.plot([settling_time], [y_assentamento], "cd", label='Tempo de acomodação (+/-2%)')


    plt.grid(True, which='both', linestyle='--', color='gray')
    plt.xlabel('Tempo, t', fontsize=12, color='black')
    plt.ylabel('Saída do sistema, y', fontsize=12, color='black')
    plt.title(f'Resposta ao Degrau', fontsize=14, color='black')
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=14, facecolor='white', edgecolor='black')
    plt.gca().set_facecolor('#f0f0f0')  # Fundo claro para o gráfico
    plt.gca().spines['bottom'].set_color('black')
    plt.gca().spines['left'].set_color('black')
    plt.tick_params(axis='x', colors='black')
    plt.tick_params(axis='y', colors='black')
    plt.show()

    # Exibindo propriedades do sistema
    output.clear_output(wait=True)
    with output:
      if fator_xi<1:
        display(HTML(f"<b>Valor Estacionário:</b> {ganho:.2f}"))
        display(HTML(f"<b>Overshoot (%):</b> {overshoot:.4f}"))
        display(HTML(f"<b>Tempo de subida:</b> {t_subida_resposta:.2f}"))
        display(HTML(f"<b>Instante de Pico:</b> {t_pico_resposta:.2f}"))
        display(HTML(f"<b>Tempo de Acomodação (+/-2%):</b> {settling_time:.2f}"))
      else:
        display(HTML(f"<b>Valor Estacionário:</b> {ganho:.2f}"))
        display(HTML(f"<b>Tempo de Acomodação (+/-2%):</b> {settling_time:.2f}"))
        display(HTML(f"<b>Instante de Pico:</b> {t_pico_resposta:.2f}"))



# Sliders com cores e descrições
K_slider = widgets.FloatSlider(min=-5, max=5, step=0.01, value=1.0,
                                 description='Ganho', style={'description_width': 'initial'},
                                 layout=Layout(width='400px'), continuous_update=False)

tau_slider = widgets.FloatSlider(min=0.01, max=5, step=0.01, value=1.0,
                                 description='Período', style={'description_width': 'initial'},
                                 layout=Layout(width='400px'), continuous_update=False)

xi_slider = widgets.FloatSlider(min=0.05, max=1.5, step=0.01, value=0.6,
                                description='Amortecimento', style={'description_width': 'initial'},
                                layout=Layout(width='400px'), continuous_update=False)

# Função para redefinir sliders
def reset_sliders(b):
    K_slider.value = 1.0
    tau_slider.value = 1.0
    xi_slider.value = 0.6

# Botão de reset com estilo
reset_button = Button(description="Redefinir Valores",
                      button_style='primary', layout=Layout(width='200px'))

reset_button.on_click(reset_sliders)

# Output para propriedades
output = Output()

# Layout completo com sliders, botão de reset e propriedades
ui = VBox([reset_button, output])

# Interatividade com os sliders já definidos
# Não estamos passando os sliders para o interact, assim ele não cria os sliders automaticamente
interact(simula_2ordem, ganho=K_slider, tau=tau_slider, fator_xi=xi_slider)

# Exibindo a interface com os sliders, botão de reset e propriedades
display(ui)


interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='Ganho', layout=Layout(width…

A seguir, explicam-se os efeitos de cada um desses parâmetros no comportamento do sistema.

1. Ganho (K)
O **ganho** está relacionado à amplitude da resposta do sistema. Ele determina o quanto a saída do sistema é amplificada ou atenuada em relação à entrada (no caso, o degrau). Aumentar o ganho resulta em uma curva de saída mais alta, já que o sistema amplifica a entrada.

2. Período (τ)
O **período** está relacionado ao tempo que o sistema leva para atingir a resposta final. Ele determina a velocidade de resposta do sistema. Com isso, aumentar o **τ** resulta em uma resposta mais lenta. A curva de saída demora mais tempo para atingir o valor final.

3. Fator de Amortecimento (ξ)
O **fator de amortecimento** controla o comportamento oscilatório do sistema. Ele determina se o sistema vai oscilar antes de se estabilizar e com que rapidez essas oscilações diminuem. Dessa forma:
  - Quando **ξ** é baixo (próximo de zero), o sistema oscila bastante antes de se estabilizar.
  - Quando **ξ** é alto (próximo de 1), o sistema se estabiliza rapidamente, sem oscilações.
  - Se **ξ** for maior que 1, o sistema se torna super-amortecido, não oscila, mas também leva mais tempo para atingir o valor final.